In [6]:
import pandas as pd
import json
import os

In [ ]:
df_lenovo = pd.read_csv('datos/lenovo/productos.csv')
df_lenovo.head()

In [2]:
def transformaciones(data_json: dict):
    productos = data_json["data"]["items"]["results"]

    df = pd.DataFrame(productos)
    df = df[['id', 'title', 'katalogCategoryId', 'brand', 'skus','slug']]
    df = pd.json_normalize(df.to_dict('records'), sep='_')
    df = df.drop(columns=['brand___typename','skus___typename'], axis=1)


    def ensure_list(v):
        return v if isinstance(v, list) else []

    def first_list(v):
        return v[0] if isinstance(v, list) and v else None

    def get_category_names(categ):
        """
        'categ' suele ser una lista de rutas (cada ruta es una lista de dicts con name/slug).
        Retorna una lista con todos los nombres de categorías.
        """
        if not isinstance(categ, list) or not categ:
            return []
        ruta = categ[0]
        if not isinstance(ruta, list):
            return []
        return [x.get("name") for x in ruta if isinstance(x, dict)]

    # ---------- 1) Exploto SKUs: una fila por SKU ----------
    tmp = df.copy()
    tmp["skus_results"] = tmp["skus_results"].apply(ensure_list)
    tmp = tmp.explode("skus_results", ignore_index=True)

    # Normalizo el dict del SKU en columnas
    sku_cols = pd.json_normalize(tmp["skus_results"]).add_prefix("sku_")
    tmp = tmp.drop(columns=["skus_results"]).join(sku_cols)

    # ---------- 2) Exploto pricing ----------
    tmp["sku_pricing"] = tmp["sku_pricing"].apply(ensure_list)
    tmp = tmp.explode("sku_pricing", ignore_index=True)
    pricing_cols = pd.json_normalize(tmp["sku_pricing"]).add_prefix("pricing_")
    tmp = tmp.drop(columns=["sku_pricing"]).join(pricing_cols)

    # ---------- 3) Priorizar canal 'fravega-ecommerce' ----------
    tmp["_pref_order"] = (tmp["pricing_channel"] != "fravega-ecommerce").astype(int)
    tmp = tmp.sort_values(["sku_code", "_pref_order"]).drop_duplicates(subset=["sku_code"], keep="first").drop(columns=["_pref_order"])

    # ---------- 4) Obtener categorías como lista ----------
    tmp["categories"] = tmp["sku_categorization"].apply(get_category_names)

    # ---------- 5) Selección final ----------
    df_skus = tmp[[
        "id",                 # ID del producto
        "sku_code",           # Código del SKU
        "categories",         # Lista de categorías
        "pricing_listPrice",  # Precio de lista
        "pricing_salePrice"   # Precio de oferta
    ]].rename(columns={
        "sku_code": "sku_id",
        "pricing_listPrice": "list_price",
        "pricing_salePrice": "sale_price"
    })

    return df.merge(df_skus, on='id').drop('skus_results', axis=1)

In [4]:
jsons = []
import os
archivos = list(f'datos/lenovo/productos/{x}' for x in os.listdir('datos/lenovo/productos/'))
archivos

['datos/lenovo/productos/productos-offset0.json',
 'datos/lenovo/productos/productos-offset100.json',
 'datos/lenovo/productos/productos-offset150.json',
 'datos/lenovo/productos/productos-offset200.json',
 'datos/lenovo/productos/productos-offset250.json',
 'datos/lenovo/productos/productos-offset300.json',
 'datos/lenovo/productos/productos-offset350.json',
 'datos/lenovo/productos/productos-offset400.json',
 'datos/lenovo/productos/productos-offset450.json',
 'datos/lenovo/productos/productos-offset50.json']

In [7]:
for archivo in archivos:
    with open(archivo, 'r', encoding='utf-8') as f:
        jsons.append(json.load(f))

In [10]:
df = pd.concat(list(transformaciones(df) for df in jsons))

In [14]:
df = df.sort_values('title').reset_index(drop=True)

In [15]:
df.to_csv('datos/lenovo/productos.csv', index=False)